In [ ]:
using Plots; immerse()
default(size=(500,300), leg=false)

# creates x/y vectors which can define a grid in a zig-zag pattern
function gridxy(lim, n::Int)
    xs = linspace(lim..., n)
    xypairs = vec([(x,y) for x in vcat(xs,reverse(xs)), y in xs])
    Plots.unzip(xypairs)
end

# The problem... can we classify the functions?

In [ ]:
# these are the functions we want to classify
scalar = 10  # larger is harder... start with 3
f1(x) = 0.6sin(scalar * x) + 0.1
f2(x) = f1(x) - 0.2

# On to the fun...

In [ ]:
# pick the plotting limits
lim = (-1,1)
funcs = [f1, f2]
n = 40
gridx, gridy = gridxy(lim, n)
default(xlim = lim, ylim = lim)

function initialize_plot(funcs, lim, gridx, gridy; kw...)
    # show the grid
    plot([gridx gridy], [gridy gridx], c=:black, kw...)

    # show the funcs
    plot!(funcs, lim..., l=(4,[:blue :red]))
end

# kick off an animation... we can save frames whenever we want, lets save the starting frame
function initialize_animation()
    anim = Animation()
    frame(anim)
    anim
end

# lets see what we're dealing with...
p = initialize_plot(funcs, lim, gridx, gridy)

# That looks tricky... lets build a neural net!

In [ ]:
using OnlineAI
net = buildTanhClassificationNet(
    2,    # number of inputs
    1,    # number of outputs
    [4,4,2], # hidden layers structure
#     params = NetParams(gradientModel = SGDModel(η=1e-5))
)

# take x matrix and convert to the first layer's activation
function activateHidden(net, x)
    @assert net.layers[end].nin == 2
    proj = zeros(nrows(x), 2)
    for i in 1:nrows(x)
        data = row(x,i)
        for layer in net.layers[1:end-1]
            OnlineAI.forward!(layer, data, false)
            data = layer.a
        end
        row!(proj, i, data)
    end
    vec(proj[:,1]), vec(proj[:,2])
end    

# Update our model and the visualization

In [ ]:
p = initialize_plot(funcs, lim, gridx, gridy)
anim = initialize_animation()
gui()

progressviz = track_progress(net)
gui(progressviz.subplt)

In [ ]:
iterations_per_frame = 2000
total_frames = 100
dist = Distributions.Uniform(lim...)

for frm in 1:total_frames
    # pick one of the functions at random, sample from the x line, then update the
    # neural net with [x, f(x)] as the inputsn = 1000
    for i in 1:iterations_per_frame
        f = sample(funcs)
        x = rand(dist)
        y = f == f1 ? 1.0 : -1.0
        update!(net, Float64[x, f(x)], [y])
    end

    # update the plot... project each series to the first hidden layer and reset the data
    # NOTE: this works because `getindex` and `setindex` are overloaded to get/set the underlying plot series data
    x = linspace(lim..., 50)
    p[1] = activateHidden(net, hcat(gridx, gridy))
    p[2] = activateHidden(net, hcat(gridy, gridx))
    p[3] = activateHidden(net, hcat(x, map(f1,x)))
    p[4] = activateHidden(net, hcat(x, map(f2,x)))

    # show/update the plot
    gui(p)
    frame(anim)
    
    # update the progress visualization
    update!(progressviz, true)
end

# Animate!

In [ ]:
gif(anim, fps = 20)

In [ ]:
# show the network (uses Qwt, visualize isn't available unless you import it)
import Qwt
viz = visualize(net);

In [ ]:
# update the net representation with weights, etc
update!(viz)

In [ ]:
gui(progressviz.subplt)

In [ ]:
p1 = plot(rand(20))
p2 = plot(rand(10))
p3 = scatter(rand(100))
p4 = plot(rand(1000))

In [ ]:
subplot(p1,p2,p3,p4, nr=1, leg=false)

In [ ]:
immerse()
p = plot(rand(10))

In [ ]:
gui()

In [ ]:
append!(p,1,rand(10))
gui()

In [ ]:
sp = progressviz.subplt.plts[1].o.widget[:minimumSizeHint]()